In [147]:
import pandas as pd
import re

import past_guest_module as pm

import sys
sys.path.append('../zoho_data_app/')

import main_module as m

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [148]:
path = r'C:\Users\fajar.fatoni\Documents\Python\Data\check_country_code_analytic.xlsx'
df_analytic = pd.read_excel(path, dtype='string', sheet_name='Sheet1')
count_data_awal = df_analytic.shape[0]
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK063375,Karma Resorts,Other,Past Guests,Karma Kandara,PF291658,Mr,.,Ryan,Male,...,2024-11-05 00:00:00,4,HOUSE_USE,V_OWNER_NRB,COMP,6,0,0,Email and Phone,NO
1,KK067750,Karma Resorts,Other,Past Guests,Karma Kandara,PF050744,Mr,Brian,Bowden,Male,...,2025-01-27 00:00:00,14,HOUSE_USE,V_OWNER_NRB,COMP,1,0,0,Email and Phone,NO
2,KK068892,Karma Resorts,Other,Past Guests,Karma Kandara,PF050744,Mr,Brian,Bowden,Male,...,2025-02-02 00:00:00,1,HOUSE_USE,V_OWNER_NRB,COMP,1,0,0,Email and Phone,NO
3,KK069234,Karma Resorts,Other,Past Guests,Karma Kandara,PF050744,Mr,Brian,Bowden,Male,...,2025-02-16 00:00:00,7,HOUSE_USE,V_OWNER_NRB,COMP,1,0,0,Email and Phone,NO
4,KK071493,Karma Resorts,Other,Past Guests,Karma Kandara,PF307075,Mr.,Sudhir,Ravi,Male,...,2025-03-31 00:00:00,5,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2687,KK067267,Karma Resorts,Other,Past Guests,Karma Kandara,PF295577,<NA>,Sourab,Pramanik,Unknown,...,2025-09-20 00:00:00,2,COMP,RESERVED,KC,1,0,0,Email and Phone,NO
2688,KK055583,Karma Resorts,Other,Past Guests,Karma Kandara,PF273108,Mr,Andrew,Ridgeway,Male,...,2024-07-14 00:00:00,2,HOUSE_USE,V_OWNER_NRB,COMP,2,0,0,Email and Phone,NO
2689,KK053272,Karma Resorts,Other,Past Guests,Karma Kandara,PF268176,Mr,Carlos,Alcario,Male,...,2024-07-22 00:00:00,4,TO,TO,RETAIL,3,2,0,Email and Phone,NO
2690,KK049831,Karma Resorts,Other,Past Guests,Karma Kandara,PF262183,Mr,Nicolas,Khatchadourian,Male,...,2024-02-11 00:00:00,1,HOUSE_USE,V_OWNER_NRB,COMP,3,1,0,Email and Phone,NO


In [149]:
df_analytic.columns

Index(['Booking_Id', 'Lead Brand', 'Lead Sub- Brand', 'Lead Source',
       'Resort_Name', 'ProfileRef', 'Salutation', 'First_Name', 'Surname',
       'Gender', 'Language', 'DOB', 'Street', 'Area', 'Town', 'County',
       'Country', 'Nationality', 'PostCode', 'TelNo', 'MobileNo', 'Email',
       'Exclude From Mailing', 'Opt In', 'Booking Status', 'DateArrival',
       'DateDepart', 'No. of Nights', 'Market_Segment', 'Media_Code',
       'Media Code Group', 'No. of Adult', 'No. of Children', 'No. of Infants',
       'Contact Type', 'Odyssey Members'],
      dtype='object')

In [150]:
df_analytic.fillna('empty', inplace=True)

In [151]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


In [152]:
m.clean_space(df_email, 'Email')
df_email['Email'] = m.lowercase(df_email, 'Email')

c:\Users\fajar.fatoni\Documents\Python\Task Offline\past_guest_toni_app\../zoho_data_app\main_module.py:127: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)


In [ ]:
df_analytic.loc[
    (df_analytic['TelNo'] == 'empty') & (df_analytic['MobileNo'] != 'empty'), 
    'TelNo'] = df_analytic['MobileNo']
df_analytic.loc[df_analytic['TelNo'] == df_analytic['MobileNo'],'MobileNo'] = 'empty'

In [154]:
# hitung jumlah duplicate
count_duplicate_general = df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])].shape[0]

df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])]

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
2,KK068892,Karma Resorts,Other,Past Guests,Karma Kandara,PF050744,Mr,Brian,Bowden,Male,...,2025-02-02 00:00:00,1,HOUSE_USE,V_OWNER_NRB,COMP,1,0,0,Email and Phone,NO
3,KK069234,Karma Resorts,Other,Past Guests,Karma Kandara,PF050744,Mr,Brian,Bowden,Male,...,2025-02-16 00:00:00,7,HOUSE_USE,V_OWNER_NRB,COMP,1,0,0,Email and Phone,NO
14,KK066659,Karma Resorts,Other,Past Guests,Karma Kandara,PF295577,empty,Sourab,Pramanik,Unknown,...,2025-04-18 00:00:00,3,COMP,RESERVED,KC,1,0,0,Email and Phone,NO
15,KK066662,Karma Resorts,Other,Past Guests,Karma Kandara,PF295577,empty,Sourab,Pramanik,Unknown,...,2025-05-17 00:00:00,2,COMP,RESERVED,KC,1,0,0,Email and Phone,NO
17,KK066666,Karma Resorts,Other,Past Guests,Karma Kandara,PF295577,empty,Sourab,Pramanik,Unknown,...,2025-04-30 00:00:00,4,COMP,RESERVED,KC,4,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2673,KK058579,Karma Resorts,Other,Past Guests,Karma Kandara,PF280744,Ms,Bianca,Raslan,Female,...,2024-12-30 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
2678,KK052502,Karma Resorts,Other,Past Guests,Karma Kandara,PF267057,Mr,.,Rahmat,Male,...,2024-04-12 00:00:00,2,HOUSE_USE,GSTAFF,COMP,3,0,0,Email and Phone,NO
2687,KK067267,Karma Resorts,Other,Past Guests,Karma Kandara,PF295577,empty,Sourab,Pramanik,Unknown,...,2025-09-20 00:00:00,2,COMP,RESERVED,KC,1,0,0,Email and Phone,NO
2689,KK053272,Karma Resorts,Other,Past Guests,Karma Kandara,PF268176,Mr,Carlos,Alcario,Male,...,2024-07-22 00:00:00,4,TO,TO,RETAIL,3,2,0,Email and Phone,NO


In [155]:
df_analytic.drop_duplicates(subset=['TelNo','MobileNo','Email'], inplace=True)
df_analytic.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = df_analytic.shape[0]

df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK063375,Karma Resorts,Other,Past Guests,Karma Kandara,PF291658,Mr,.,Ryan,Male,...,2024-11-05 00:00:00,4,HOUSE_USE,V_OWNER_NRB,COMP,6,0,0,Email and Phone,NO
1,KK067750,Karma Resorts,Other,Past Guests,Karma Kandara,PF050744,Mr,Brian,Bowden,Male,...,2025-01-27 00:00:00,14,HOUSE_USE,V_OWNER_NRB,COMP,1,0,0,Email and Phone,NO
2,KK071493,Karma Resorts,Other,Past Guests,Karma Kandara,PF307075,Mr.,Sudhir,Ravi,Male,...,2025-03-31 00:00:00,5,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
3,KK048506,Karma Resorts,Other,Past Guests,Karma Kandara,PF259954,Mr,Vipin,Haripuram,Male,...,2024-04-01 00:00:00,2,COMP,STAFF,COMP,4,0,0,Email and Phone,NO
4,KK070933,Karma Resorts,Other,Past Guests,Karma Kandara,PF306083,Miss,Yumeng,Li,Female,...,2025-03-12 00:00:00,2,OTA,OTA,RETAIL,4,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,KK068891,Karma Resorts,Other,Past Guests,Karma Kandara,PF301826,Mr,Liu,Hao,Male,...,2025-02-01 00:00:00,4,HOUSE_USE,V_OWNER_NRB,COMP,3,0,0,Email and Phone,NO
2020,KK047513,Karma Resorts,Other,Past Guests,Karma Kandara,PF258233,Ms,Lorraine Karen,Sarayeldin,Female,...,2024-01-06 00:00:00,2,HOUSE_USE,V_OWNER_NRB,COMP,3,0,0,Email and Phone,NO
2021,KK067755,Karma Resorts,Other,Past Guests,Karma Kandara,PF299622,Mr,Dorian,Pryce,Male,...,2025-01-14 00:00:00,2,COMP,GSTAFF,COMP,1,0,0,Email and Phone,NO
2022,KK055583,Karma Resorts,Other,Past Guests,Karma Kandara,PF273108,Mr,Andrew,Ridgeway,Male,...,2024-07-14 00:00:00,2,HOUSE_USE,V_OWNER_NRB,COMP,2,0,0,Email and Phone,NO


In [156]:
find_member_booking = df_analytic.copy()

find_member_booking.loc[find_member_booking['TelNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['MobileNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'
find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

find_member_booking.replace('empty', '', inplace=True)

In [157]:
find_member_booking.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\cek_member_zoho_analytic.csv', index=False)

In [158]:
count_dedup_member = df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])].shape[0]


df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])]


,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
11,KK066554,Karma Resorts,Other,Past Guests,Karma Kandara,PF295577,empty,Sourab,Pramanik,Unknown,...,2025-02-04 00:00:00,3,COMP,RESERVED,KC,2,0,0,Email and Phone,NO
13,KK066542,Karma Resorts,Other,Past Guests,Karma Kandara,PF297585,empty,Shiju,Shaji,Unknown,...,2025-01-04 00:00:00,2,COMP,RESERVED,KC,1,0,0,Email and Phone,NO
20,KK065084,Karma Resorts,Other,Past Guests,Karma Kandara,PF215766,Mrs,Sharon,Chandra,Female,...,2025-01-01 00:00:00,11,HOUSE_USE,V_OWNER,COMP,4,0,0,Email and Phone,NO
30,KK046214,Karma Resorts,Other,Past Guests,Karma Kandara,PF076130,Mr,Laurent,Combalot,Male,...,2024-04-07 00:00:00,9,HOUSE_USE,V_OWNER,COMP,4,0,0,Email and Phone,NO
45,KK072843,Karma Resorts,Other,Past Guests,Karma Kandara,PF076238,Mrs,Grace Alexandra,Tirtha,Female,...,2025-04-15 00:00:00,3,HOUSE_USE,V_OWNER,COMP,6,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1957,KK063238,Karma Resorts,Other,Past Guests,Karma Kandara,PF291381,Mrs,Sharon,Andrew,Female,...,2025-01-13 00:00:00,7,DIGITAL,GHPA,DIGITAL,2,1,0,Email and Phone,NO
1959,KK067018,Karma Resorts,Other,Past Guests,Karma Kandara,PF298254,empty,Sruthi,Sunil,Unknown,...,2025-02-03 00:00:00,2,COMP,RESERVED,KC,1,0,0,Email Only,NO
1966,KK070321,Karma Resorts,Other,Past Guests,Karma Kandara,PF304803,Mrs,A,A,Female,...,2025-02-27 00:00:00,5,DIGITAL,GHPA,DIGITAL,2,0,0,Email and Phone,NO
1996,KK053261,Karma Resorts,Other,Past Guests,Karma Kandara,PF250838,Frau,Deborah,Hillmann,Female,...,2024-05-05 00:00:00,4,HOUSE_USE,GER_OWE,COMP,3,0,0,Email and Phone,NO


In [159]:
df_analytic.drop(df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])
].index, inplace=True)

df_analytic.reset_index(drop=True, inplace=True)

count_after_dedup_member = df_analytic.shape[0]

In [160]:
count_data_akhir = df_analytic.shape[0]

In [161]:
df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK047166,Karma Resorts,Other,Past Guests,Karma Kandara,PF257369,Mrs,Angelique,Summers,Female,...,2024-01-02 00:00:00,1,OTA,OTA,RETAIL,4,0,0,Phone Only,NO
1,KK047164,Karma Resorts,Other,Past Guests,Karma Kandara,PF257367,Mrs,Kironi,Sciiim,Female,...,2024-01-02 00:00:00,1,OTA,OTA,RETAIL,6,0,0,Phone Only,NO
2,KK047359,Karma Resorts,Other,Past Guests,Karma Kandara,PF257934,Mr,Raghav,Kacker,Male,...,2024-01-02 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
3,KK047316,Karma Resorts,Other,Past Guests,Karma Kandara,PF257747,empty,Nicolas,De Tesla Junior,Unknown,...,2024-01-02 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
4,KK047172,Karma Resorts,Other,Past Guests,Karma Kandara,PF257374,Mr,Supachai,Yakomphor,Male,...,2024-01-03 00:00:00,1,OTA,OTA,RETAIL,8,0,0,Phone Only,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1925,KK071685,Karma Resorts,Other,Past Guests,Karma Kandara,PF307526,Miss,Megan,Casson,Female,...,2025-10-31 00:00:00,6,DIGITAL,GHPA,DIGITAL,2,0,0,Email and Phone,NO
1926,KK064587,Karma Resorts,Other,Past Guests,Karma Kandara,PF293968,Mr,Michael,Robinson,Male,...,2025-11-03 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
1927,KK071694,Karma Resorts,Other,Past Guests,Karma Kandara,PF307558,Ms,Irina,Shelest,Female,...,2025-11-24 00:00:00,2,OTA,OTA,RETAIL,4,0,0,Phone Only,NO
1928,KK069741,Karma Resorts,Other,Past Guests,Karma Kandara,PF303681,Mr,Shazmin,Mohideen,Male,...,2025-12-05 00:00:00,5,DIGITAL,GHPA,DIGITAL,2,2,0,Email and Phone,NO


In [162]:
df_analytic['DOB'].loc[df_analytic['DOB']=='']

Series([], Name: DOB, dtype: string)

In [163]:
df_analytic['DOB'].replace('empty', '', inplace=True)
dob_con_date = pd.to_datetime(df_analytic['DOB'], errors='coerce')
dob_date = dob_con_date.dt.strftime('%Y-%m-%d')

arr_con_date = pd.to_datetime(df_analytic['DateArrival'])
arr_date = arr_con_date.dt.strftime('%Y-%m-%d')

dep_con_date = pd.to_datetime(df_analytic['DateDepart'])
dep_date = dep_con_date.dt.strftime('%Y-%m-%d')

df_analytic['DOB'] = dob_date
df_analytic['DateArrival'] = arr_date
df_analytic['DateDepart'] = dep_date


C:\Users\fajar.fatoni\AppData\Local\Temp\ipykernel_9984\1513714307.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_analytic['DOB'].replace('empty', '', inplace=True)


In [164]:
df_analytic.replace('empty', '', inplace=True)

df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK047166,Karma Resorts,Other,Past Guests,Karma Kandara,PF257369,Mrs,Angelique,Summers,Female,...,2024-01-02,1,OTA,OTA,RETAIL,4,0,0,Phone Only,NO
1,KK047164,Karma Resorts,Other,Past Guests,Karma Kandara,PF257367,Mrs,Kironi,Sciiim,Female,...,2024-01-02,1,OTA,OTA,RETAIL,6,0,0,Phone Only,NO
2,KK047359,Karma Resorts,Other,Past Guests,Karma Kandara,PF257934,Mr,Raghav,Kacker,Male,...,2024-01-02,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
3,KK047316,Karma Resorts,Other,Past Guests,Karma Kandara,PF257747,,Nicolas,De Tesla Junior,Unknown,...,2024-01-02,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
4,KK047172,Karma Resorts,Other,Past Guests,Karma Kandara,PF257374,Mr,Supachai,Yakomphor,Male,...,2024-01-03,1,OTA,OTA,RETAIL,8,0,0,Phone Only,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1925,KK071685,Karma Resorts,Other,Past Guests,Karma Kandara,PF307526,Miss,Megan,Casson,Female,...,2025-10-31,6,DIGITAL,GHPA,DIGITAL,2,0,0,Email and Phone,NO
1926,KK064587,Karma Resorts,Other,Past Guests,Karma Kandara,PF293968,Mr,Michael,Robinson,Male,...,2025-11-03,3,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
1927,KK071694,Karma Resorts,Other,Past Guests,Karma Kandara,PF307558,Ms,Irina,Shelest,Female,...,2025-11-24,2,OTA,OTA,RETAIL,4,0,0,Phone Only,NO
1928,KK069741,Karma Resorts,Other,Past Guests,Karma Kandara,PF303681,Mr,Shazmin,Mohideen,Male,...,2025-12-05,5,DIGITAL,GHPA,DIGITAL,2,2,0,Email and Phone,NO


In [165]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Data Setelah Script Past Guest Sebelumnya : "+str(count_data_awal))
print("Jumlah Data Yang Duplicate : "+str(count_duplicate_general))
print("Jumlah Data Setelah Hapus Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Data Yang Termasuk Member: "+str(count_dedup_member))
print("Jumlah Data Setelah Hapus Member : "+str(count_after_dedup_member))
print("Jumlah Data Akhir : "+str(count_data_akhir))

Summary
Jumlah Data Setelah Script Past Guest Sebelumnya : 2692
Jumlah Data Yang Duplicate : 668
Jumlah Data Setelah Hapus Duplicate : 2024
Jumlah Data Yang Termasuk Member: 94
Jumlah Data Setelah Hapus Member : 1930
Jumlah Data Akhir : 1930


In [166]:
df_analytic.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\final_analytics.xlsx', index=False)